In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("datasnaek/chess")

print("Path to dataset files:", path)

c:\Users\Karolcia\anaconda3\envs\AI_2025\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Karolcia\.cache\kagglehub\datasets\datasnaek\chess\versions\1


In [40]:
import pandas as pd
import numpy as np
import pyspark as psp
import os
import ast
import seaborn as sns
import matplotlib.pyplot as plt

dataset_path = os.path.join(path,'games.csv')
df = pd.read_csv(dataset_path)##, delimiter="\t")
df = pd.DataFrame(df)

In [41]:
df.head(5)

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              20058 non-null  object 
 1   rated           20058 non-null  bool   
 2   created_at      20058 non-null  float64
 3   last_move_at    20058 non-null  float64
 4   turns           20058 non-null  int64  
 5   victory_status  20058 non-null  object 
 6   winner          20058 non-null  object 
 7   increment_code  20058 non-null  object 
 8   white_id        20058 non-null  object 
 9   white_rating    20058 non-null  int64  
 10  black_id        20058 non-null  object 
 11  black_rating    20058 non-null  int64  
 12  moves           20058 non-null  object 
 13  opening_eco     20058 non-null  object 
 14  opening_name    20058 non-null  object 
 15  opening_ply     20058 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(9)
memory usage: 2.3+ MB


In [43]:
df['created_at'] = pd.to_datetime(df['created_at'].astype('Int64'), unit='ms', errors='coerce')
df['last_move_at'] = pd.to_datetime(df['last_move_at'].astype('Int64'), unit='ms', errors='coerce')



#df.head(5)
df.sort_values('created_at',ascending=False).head(5)

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
19462,mLY9CCtH,True,2017-09-04 02:45:43.790,2017-09-04 02:57:07.262,65,resign,white,10+0,vertis9016,1822,samoo1477,1767,d4 e6 e4 d5 e5 c5 c3 Qb6 Nf3 Nc6 Bd3 cxd4 cxd4...,C02,French Defense: Advance Variation | Milner-Ba...,11
19463,dGvzzC9L,True,2017-09-04 02:30:59.427,2017-09-04 02:45:16.572,67,resign,white,10+0,samoo1477,1754,vertis9016,1835,e4 e6 d4 d5 Nd2 Ne7 c3 dxe4 Nxe4 Nf5 Nf3 Bd6 B...,C03,French Defense: Tarrasch Variation,5
16581,8Xr8Hmp7,True,2017-09-04 01:41:20.676,2017-09-04 01:48:47.504,48,resign,black,10+0,elijahreese,1873,darkwasthenight,1913,e4 e5 Nf3 f5 d3 Nf6 exf5 Nc6 Bg5 d5 Nc3 Be7 Be...,C40,Latvian Gambit,4
16582,5JxE5a6e,True,2017-09-04 01:28:53.111,2017-09-04 01:41:01.184,61,mate,white,10+0,elijahreese,1871,joseroldan,1456,e4 d6 d4 g6 c4 Bg7 Nc3 Nd7 Nf3 b6 Bf4 Bb7 Bd3 ...,A42,Modern Defense: Averbakh Variation,7
13682,qCmTWffu,True,2017-09-04 01:27:41.297,2017-09-04 01:32:23.757,55,mate,white,10+0,networkchess2,1940,tatianelongato,1847,d4 c6 Nf3 d5 c4 Nf6 Bg5 e6 e3 Be7 Nc3 O-O Bd3 ...,D11,Slav Defense: Bonet Gambit,7


In [45]:
df[(df['white_id'] == 'elijahreese') | (df['black_id'] == 'elijahreese')].head(15)

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
16581,8Xr8Hmp7,True,2017-09-04 01:41:20.676,2017-09-04 01:48:47.504,48,resign,black,10+0,elijahreese,1873,darkwasthenight,1913,e4 e5 Nf3 f5 d3 Nf6 exf5 Nc6 Bg5 d5 Nc3 Be7 Be...,C40,Latvian Gambit,4
16582,5JxE5a6e,True,2017-09-04 01:28:53.111,2017-09-04 01:41:01.184,61,mate,white,10+0,elijahreese,1871,joseroldan,1456,e4 d6 d4 g6 c4 Bg7 Nc3 Nd7 Nf3 b6 Bf4 Bb7 Bd3 ...,A42,Modern Defense: Averbakh Variation,7
16583,20OmhieD,True,2017-09-04 01:26:00.932,2017-09-04 01:27:49.943,13,resign,white,10+0,elijahreese,1871,radster859,1098,e4 e5 Bc4 Nf6 d3 Nc6 Nf3 Nd4 Nxe5 Bb4+ c3 Bd6 ...,C55,Italian Game: Two Knights Defense | Modern Bi...,7
16584,8PLe6htc,True,2017-09-03 03:24:09.285,2017-09-03 03:35:26.185,80,resign,white,10+0,segentaralam,1836,elijahreese,1884,e4 e5 Nf3 Nc6 Bb5 a6 Ba4 d6 d4 exd4 Nxd4 Bd7 N...,C71,Ruy Lopez: Morphy Defense | Modern Steinitz D...,8
16585,CmL3yjXT,True,2017-09-03 03:07:30.729,2017-09-03 03:21:21.509,101,outoftime,white,10+0,jeandossantos,1822,elijahreese,1886,d4 d5 Nf3 Nc6 e3 Nf6 Bd3 e6 O-O Bd6 Re1 O-O a3...,D02,Queen's Pawn Game: Chigorin Variation,4
16586,HSGAIrXF,True,2017-09-03 02:56:26.329,2017-09-03 03:07:04.651,68,resign,black,10+0,elijahreese,1901,segentaralam,1806,e4 e5 Nf3 Nc6 c3 Nf6 d3 h6 Be3 d5 exd5 Nxd5 Be...,C44,Ponziani Opening: Jaenisch Counterattack,6
16587,R1K9ltRP,True,2017-09-03 02:49:01.543,2017-09-03 02:55:42.562,87,resign,white,10+0,elijahreese,1893,jhonwalter,1768,e4 e6 c4 c6 Nc3 Bb4 Nf3 Bxc3 dxc3 Nf6 Bg5 O-O ...,C00,French Defense: Steiner Variation,3
16588,bv1UoisS,True,2017-09-03 02:33:49.042,2017-09-03 02:48:01.072,104,mate,black,10+0,mrtiniano,1738,elijahreese,1886,c4 c5 e3 Nc6 d4 cxd4 exd4 d5 b3 dxc4 d5 Nb4 bx...,A30,English Opening: Symmetrical Variation,2
16589,EK64Avgy,True,2017-09-03 02:29:00.767,2017-09-03 02:32:53.122,81,resign,white,10+0,elijahreese,1872,carvaflu,1940,e4 Nc6 d4 d5 exd5 Qxd5 Nf3 Bg4 Nc3 Qe6+ Qe2 Bx...,B00,Nimzowitsch Defense: Scandinavian Variation | ...,6
16590,jb6MsLP4,True,2017-09-03 02:26:56.772,2017-09-03 02:28:16.677,21,resign,white,10+0,elijahreese,1862,sely-chess,1809,e4 e5 Nf3 Nc6 c3 Bc5 d4 exd4 cxd4 Bb6 Be3 d5 e...,C44,Ponziani Opening,5


In [49]:
# df_white = df[['white_id', 'white_rating']]
# df_black = df[['black_id', 'black_rating']]

# df_white = df_white.rename(columns={'white_id': 'player_id'})
# df_black = df_black.rename(columns={'black_id': 'player_id'})

# players = pd.merge(df_white, df_black, on='player_id', how='outer')
# players = players.drop_duplicates(subset=['player_id'])
# players.sort_values(by=['white_rating'], ascending=False).head(10)

In [50]:
# players['rating_sum'] = players['black_rating']+players['white_rating']
# players.sort_values(by=['rating_sum'], ascending=False).head(5)

In [56]:
df['higher_score'] = df.apply(lambda row: 'white' if row['white_rating'] > row['black_rating'] else 'black', axis=1)
df['higher_win'] = df.apply(lambda x: 1 if x['winner'] ==  x['higher_score'] else 0, axis=1)

df.describe()

,created_at,last_move_at,turns,white_rating,black_rating,opening_ply,higher_win
count,20058,20058,20058.000000,20058.000000,20058.000000,20058.000000,20058.000000
mean,2017-01-05 11:47:32.629000,2017-01-05 12:02:02.336000,60.465999,1596.631868,1588.831987,4.816981,0.620550
min,2013-08-17 20:33:53.173000,2013-08-17 20:37:43.841000,1.000000,784.000000,789.000000,1.000000,0.000000
25%,2016-10-27 05:51:40,2016-10-27 05:51:40,37.000000,1398.000000,1391.000000,3.000000,0.000000
50%,2017-05-28 22:20:00,2017-05-28 22:20:00,55.000000,1567.000000,1562.000000,4.000000,1.000000
75%,2017-08-19 19:13:20,2017-08-19 19:13:20,79.000000,1793.000000,1784.000000,6.000000,1.000000
max,2017-09-04 02:45:43.790000,2017-09-04 02:57:07.262000,349.000000,2700.000000,2723.000000,28.000000,1.000000
std,NaN,NaN,33.570585,291.253376,291.036126,2.797152,0.485262


In [67]:
df.groupby('victory_status')[['higher_win']].agg(['sum', 'count'])

higher_win       
                      sum  count
victory_status                  
draw                    0    906
mate                 4222   6325
outoftime             999   1680
resign               7226  11147

In [68]:
df.groupby('higher_win')[['higher_win']].agg(['sum', 'count'])

higher_win       
                  sum  count
higher_win                  
0                   0   7611
1               12447  12447